In [2]:
# You only need to run this once per machine
!pip install -q -U datasets scipy ipywidgets matplotlib
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

### 0. Accelerator

Set up the Accelerator. I'm not sure if we really need this for a QLoRA given its [description](https://huggingface.co/docs/accelerate/v0.19.0/en/usage_guides/fsdp) (I have to read more about it) but it seems it can't hurt, and it's helpful to have the code for future reference. You can always comment out the accelerator if you want to try without.

In [2]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

### 1. Load Dataset

#### Preparing data

To prepare your dataset for loading, all you need is a `.jsonl` file structured something like this:
```
{"input": "What color is the sky?", "output": "The sky is blue."}
{"input": "Where is the best place to get cloud GPUs?", "output": "Brev.dev"}
```

If you choose to model your data as input/output pairs, you'll want to use something like the second `formatting_func` below, which will will combine all your features into one input string.


In [3]:
from datasets import load_dataset

train_dataset = load_dataset('json', data_files='../../Data/Training_Data/WeeveIE_Wikipedia/WeeveLVL4_J.json', split='train')  
eval_dataset = load_dataset('json', data_files='../../Data/Training_Data/WeeveIE_Wikipedia/WeeveLVL3_J.json', split='train')

### Formatting prompts

In [4]:
def formatting_func(example):
    text = f"{example['input']}"
    return text

### 2. Load Base Model

Let's now load Llama 2 7B - `meta-llama/Llama-2-7b-hf` - using 4-bit quantization!

In [5]:
# !huggingface-cli login

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "meta-llama/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 3. Tokenization

Set up the tokenizer. Add padding on the left as it [makes training use less memory](https://ai.stackexchange.com/questions/41485/while-fine-tuning-a-decoder-only-llm-like-llama-on-chat-dataset-what-kind-of-pa).

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

In [8]:
max_length = 256 # This was an appropriate max length for my dataset

def generate_and_tokenize_prompt(prompt):
    result = tokenizer(
        formatting_func(prompt),
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [9]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/212 [00:00<?, ? examples/s]

Map:   0%|          | 0/177 [00:00<?, ? examples/s]

Check that `input_ids` is padded on the left with the `eos_token` (2) and there is an `eos_token` 2 added to the end, and the prompt starts with a `bos_token` (1).

In [10]:
print(tokenized_train_dataset[1]['input_ids'])

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 29871, 278, 16547, 540, 12355, 267, 694, 341, 1008, 347, 988, 2686, 338, 29892, 763, 278, 11751, 4671, 2496, 7226, 29906, 29946, 29962, 13, 13, 28173, 5007, 278, 29382, 29892, 12931, 19952, 29892, 376, 4178, 393, 5347, 29892, 306, 3282, 29915, 29873, 8709, 1532, 23382, 363, 592, 262, 2186, 429, 2232, 472, 12855, 29889, 1932, 306, 28996, 5377, 304, 6505, 278, 379, 6727, 295, 29892, 306, 1033, 1074, 278, 11751, 4671, 2496, 29889, 306, 4689, 5007, 278, 29382, 1156, 1641, 367, 29872, 513, 10857, 29873, 491, 278, 2114, 393, 14259, 679, 304, 1074, 278, 263, 2475, 1608, 8879, 14761, 373, 1425, 311, 10819, 526, 269, 1428, 1646, 29889, 313, 17245, 29892, 29897, 372, 471, 15761, 11280, 278, 1394, 291, 29915, 29879, 350, 2152, 15741, 278, 11751, 4671, 2496, 1213, 29961, 29906, 29945, 29962, 940, 2715, 393, 376, 3664, 5595, 3721, 363, 5917, 911, 470, 5360, 29892, 306, 22151, 9442, 278, 29382, 7415, 263, 5305, 363, 1906, 1058, 526, 12534, 26902, 411, 413, 

### How does the base model do?

In [18]:
eval_prompt = "Write a story about a sailor. " # End of sentence, we just want to see what is the output

In [19]:
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0], skip_special_tokens=True))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Write a story about a sailor.  everybody's got a story to tell
A friend of mine was telling me about a story he'd written. It was a short story, but it was also a memoir, because it was about a particular time in his life. He'd never written a memoir before, but he'd been writing short stories for a long time.
He'd never written a memoir before, but he'd been writing short stories for a long time.
I was surprised that he'd never written a memoir before.
I was surprised that he'd never written a memoir before. It seemed like he'd been writing short stories for a long time.
I was surprised that he'd never written a memoir before. It seemed like he'd been writing short stories for a long time. I was surprised that he'd never written a memoir before. It seemed like he'd been writing short stories for a long time.
I was surprised that he'd never written a memoir before. It seemed like he'd been writing short stories for a long time. I was surprised that he'd never written a memoir before. I

### 4. Set Up LoRA

Now, to start our fine-tuning, we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [13]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [16]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
# print_trainable_parameters(model)

# Apply the accelerator. You can comment this out to remove the accelerator.
model = accelerator.prepare_model(model)

trainable params: 81108992 || all params: 3581521920 || trainable%: 2.264651559077991


### 5. WANDB


Let's use Weights & Biases to track our training metrics. You'll need to apply an API key when prompted. Feel free to skip this if you'd like, and just comment out the `wandb` parameters in the `Trainer` definition below.

In [17]:
# make an account at wandb.ai -> i used github to login -> in user settings api key can be generated, make that available before running this cell, i want able to shut down the prompt without a real key

!pip install -q wandb -U

import wandb, os
wandb.login()

wandb_project = "whole-text-denglish-finetune"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: powerkrieger. Use `wandb login --relogin` to force relogin


### 5. Run Training!

I didn't have a lot of training samples, so I used only 500 steps. I found that the end product worked well.

A note on training. You can set the `max_steps` to be high initially, and examine at what step your model's performance starts to degrade. There is where you'll find a sweet spot for how many steps to perform. For example, say you start with 1000 steps, and find that at around 500 steps the model starts overfitting - the validation loss goes up (bad) while the training loss goes down significantly, meaning the model is learning the training set really well, but is unable to generalize to new datapoints. Therefore, 500 steps would be your sweet spot, so you would use the `checkpoint-500` model repo in your output dir (`llama2-7b-journal-finetune`) as your final model in step 6 below.

You can interrupt the process via Kernel -> Interrupt Kernel in the top nav bar once you realize you didn't need to train anymore.

In [20]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    print("more than 1 ...")
    model.is_parallelizable = True
    model.model_parallel = True

In [26]:
import transformers
from datetime import datetime

project = "whole-text-denglish-finetune"
base_model_name = "llama2-7b"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={'use_reentrant':False},
        max_steps=500,
        learning_rate=2.5e-5, # Want a small lr for finetuning
        bf16=True,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=50,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=50,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss,Validation Loss
50,No log,2.663908
100,No log,2.613480
150,No log,2.867810
200,No log,2.856420
250,No log,3.040389
300,No log,3.074737
350,No log,3.229356
400,No log,3.216998
450,No log,3.421565
500,0.730200,3.379496


Checkpoint destination directory ./llama2-7b-whole-text-denglish-finetune/checkpoint-50 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
Checkpoint destination directory ./llama2-7b-whole-text-denglish-finetune/checkpoint-100 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
Checkpoint destination directory ./llama2-7b-whole-text-denglish-finetune/checkpoint-

TrainOutput(global_step=500, training_loss=0.73017919921875, metrics={'train_runtime': 1071.0475, 'train_samples_per_second': 0.934, 'train_steps_per_second': 0.467, 'total_flos': 1.0273463205888e+16, 'train_loss': 0.73017919921875, 'epoch': 4.72})

### 6. Drum Roll... Try the Trained Model!

It's a good idea to kill the current process so that you don't run out of memory loading the base model again on top of the model we just trained. Go to `Kernel > Restart Kernel` or kill the process via the Terminal (`nvidia smi` > `kill [PID]`). 

By default, the PEFT library will only save the QLoRA adapters, so we need to first load the base Llama 2 7B model from the Huggingface Hub:


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "meta-llama/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Llama 2 7B, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Now load the QLoRA adapter from the appropriate checkpoint directory, i.e. the best performing model checkpoint:

In [2]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "llama2-7b-whole-text-denglish-finetune/checkpoint-500")

and run your inference!

Let's try the same `eval_prompt` and thus `model_input` as above, and see if the new finetuned model performs better.

In [3]:
eval_prompt = "Write a story about a sailor. "
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=256)[0], skip_special_tokens=True))

Write a story about a sailor.  In the Geschichte of Russian Menschen, the sailor's life was normalig accompanied with viele hardships und disasters. The peasants, who started ihre service in the Russian Navy, were meist deprived of ihre peasants' privileges, oft left on ihre own in the weltweit, und were exposed to viele dangers und epidemics. In ihre mind, the Dienstleistung was well worth it, as sie were paid mehr than the peasants, und received viele benefits und privileges.

The first Russian Man in the Naval Aktivität, Ivan Moskvitin, was a Diener of Yermak und took Teil in the Safari to the Urals. In the Russian Imperium, the Naval forces were not well developed until Peter I. He not nur increased the naval forces und built Siedlung to secure the Norden coastline, but auch organized the first Russian naval Aktion against the Turks in the Mediterranean. Peter I auch organized the first Russian Naval akademie and founded the Russian Naval Mediathek.

The Russian Navy was one of the

### Sweet... it worked! The fine-tuned model now prints out journal entries in my style!

How funny to see it write like me as an angsty teenager.

I hope you enjoyed this tutorial on fine-tuning Llama 2 on your own data. If you have any questions, feel free to reach out to me on [X](https://x.com/harperscarroll) or [Discord](https://discord.gg/NVDyv7TUgJ).

🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙 🤙